# Sensitivity analysis on previous work for 30°, 45° and 60° data

In [1]:
import numpy as np
import os
import sys
from six.moves import cPickle
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils

#NEW--------------------
import sys
from sklearn.metrics import classification_report,confusion_matrix
import csv
from keras.models import load_model
#-----------------------

Using TensorFlow backend.


In [0]:
# Needed as originally code was for theano backend but now using tensor flow
from keras import backend as K
K.set_image_dim_ordering('th')

In [3]:
# Check if using CoLab or local pc and set file path accordingly
if (os.getcwd() == '/content'):
    from google.colab import drive
    drive.mount('/content/gdrive')
    FILE_PATH = '/content/gdrive/My Drive/Level-4-Project/data/'
    
else:
    FILE_PATH = os.getcwd()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
def load_batch(fpath, label_key='labels'):
    f = open(fpath, 'rb')
    if sys.version_info < (3,):
        d = cPickle.load(f)
    else:
        d = cPickle.load(f, encoding="bytes")
        # decode utf8
        for k, v in d.items():
            # added check as otherwise tries to decode a string
            if type(k) is not str:
                del(d[k])
                d[k.decode("utf8")] = v
    f.close()
    data = d["data"]
    labels = d[label_key]

    data = data.reshape(data.shape[0], 1, 75, 75)
    return data, labels

In [0]:
def load_data(data_folder):
    path = FILE_PATH + data_folder

    nb_train_samples = 34260

    X_train = np.zeros((nb_train_samples, 1, 75, 75), dtype="uint8")
    y_train = np.zeros((nb_train_samples,), dtype="uint8")

    for i in range(1, 6):
        fpath = os.path.join(path, 'data_batch_' + str(i))
        data, labels = load_batch(fpath)
        X_train[(i-1)*6852:i*6852, :, :, :] = data
        y_train[(i-1)*6852:i*6852] = labels

    fpath = os.path.join(path, 'test_batch')
    X_test, y_test = load_batch(fpath)

    y_train = np.reshape(y_train, (len(y_train), 1))
    y_test = np.reshape(y_test, (len(y_test), 1))

    return (X_train, y_train), (X_test, y_test)

In [0]:
def make_model():
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), padding='same', input_shape=(1, img_rows, img_cols), activation='relu'))
    model.add(Convolution2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Convolution2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    return model

In [0]:
data_folders = ['cifar_30deg', 'cifar_45deg', 'cifar_60deg']

In [0]:
batch_size = 100
nb_classes = 6
# nb_epoch = 100
nb_epoch = 5
data_augmentation = False

# input image dimensions
img_rows, img_cols = 75, 75
# the CIFAR10 images are RGB

In [9]:
for data_folder in data_folders:
    # the data, shuffled and split between train and test sets
    (X_train, y_train), (X_test, y_test) = load_data(data_folder)
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)
    
    model = make_model()
    
    # let's train the model using SGD + momentum (how original).
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    if not data_augmentation:
        print('Not using data augmentation.')
        hist = model.fit(X_train, Y_train,
                  batch_size=batch_size,
                  epochs=nb_epoch,
                  validation_data=(X_test, Y_test),
                  shuffle=True)

        y_pred = model.predict_classes(X_test)
        print(y_pred)
        target_names = ['ArmFasterTowards', 'ArmSlowerTowards', 'CirclingArm', 'Clapping', 'PickingUp', 'Sitting']
        #     sys.stdout = open('/home/aleksandar/sets/cifar_30deg/results/report_30deg.txt', "w")
        print("CLASSIFICATION REPORT:")
        print(classification_report(np.argmax(Y_test,axis=1), y_pred,target_names=target_names))
        print ("\n")
        print("CONFUSION MATRIX:")
        print(confusion_matrix(np.argmax(Y_test,axis=1), y_pred))

        #     with open('/home/aleksandar/sets/cifar_30deg/results/graphs_30deg.csv', "w") as f:
        #        w = csv.writer(f)
        #        train_loss = hist.history['loss']
        #        val_loss = hist.history['val_loss']
        #        train_acc = hist.history['acc']
        #        val_acc = hist.history['val_acc']
        #        helplist = list(zip(train_loss, train_acc, val_loss, val_acc))
        #        w.writerow("train_loss, train_acc, val_loss, val_acc")
        #        for val in helplist:
        #           w.writerow([val])


        #     model.save('/home/aleksandar/sets/cifar_30deg/results/model_30deg.h5')
        #-----------------------

    else:
        print('Using real-time data augmentation.')

        # this will do preprocessing and realtime data augmentation
        datagen = ImageDataGenerator(
          featurewise_center=False,  # set input mean to 0 over the dataset
          samplewise_center=False,  # set each sample mean to 0
          featurewise_std_normalization=False,  # divide inputs by std of the dataset
          samplewise_std_normalization=False,  # divide each input by its std
          zca_whitening=False,  # apply ZCA whitening
          rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
          width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
          height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
          horizontal_flip=True,  # randomly flip images
          vertical_flip=False)  # randomly flip images

        # compute quantities required for featurewise normalization
        # (std, mean, and principal components if ZCA whitening is applied)
        datagen.fit(X_train)

        # fit the model on the batches generated by datagen.flow()
        hist = model.fit_generator(datagen.flow(X_train, Y_train,
                          batch_size=batch_size),
                          samples_per_epoch=X_train.shape[0],
                          nb_epoch=nb_epoch,
                          validation_data=(X_test, Y_test))

        y_pred = model.predict_classes(X_test)
        print(y_pred)
        target_names = ['ArmFasterTowards', 'ArmSlowerTowards', 'CirclingArm', 'Clapping', 'PickingUp', 'Sitting']
        #     sys.stdout = open('/home/aleksandar/sets/cifar_30deg/results/report_30deg.txt', "w")
        print("CLASSIFICATION REPORT:")
        print(classification_report(np.argmax(Y_test,axis=1), y_pred,target_names=target_names))
        print ("\n")
        print("CONFUSION MATRIX:")
        print(confusion_matrix(np.argmax(Y_test,axis=1), y_pred))

        #     with open('/home/aleksandar/sets/cifar_30deg/results/graphs_30deg.csv', "w") as f:
        #        w = csv.writer(f)
        #        train_loss = hist.history['loss']
        #        val_loss = hist.history['val_loss']
        #        train_acc = hist.history['acc']
        #        val_acc = hist.history['val_acc']
        #        helplist = list(zip(train_loss, train_acc, val_loss, val_acc))
        #        w.writerow("train_loss, train_acc, val_loss, val_acc")
        #        for val in helplist:
        #           w.writerow([val])


        #     model.save('/home/aleksandar/sets/cifar_30deg/results/model_30deg.h5')

X_train shape: (34260, 1, 75, 75)
34260 train samples
6852 test samples
Not using data augmentation.
Train on 34260 samples, validate on 6852 samples
Epoch 1/5
34260/34260 [==============================] - 34s 999us/step - loss: 0.5631 - acc: 0.7792 - val_loss: 0.5177 - val_acc: 0.8205
Epoch 2/5
34260/34260 [==============================] - 32s 937us/step - loss: 0.0996 - acc: 0.9667 - val_loss: 0.3526 - val_acc: 0.8653
Epoch 3/5
34260/34260 [==============================] - 32s 933us/step - loss: 0.0512 - acc: 0.9843 - val_loss: 0.5214 - val_acc: 0.8269
Epoch 4/5
34260/34260 [==============================] - 32s 933us/step - loss: 0.0332 - acc: 0.9886 - val_loss: 0.4253 - val_acc: 0.8634
Epoch 5/5
34260/34260 [==============================] - 32s 933us/step - loss: 0.0252 - acc: 0.9916 - val_loss: 0.3737 - val_acc: 0.8621
[4 2 1 ... 3 5 4]
CLASSIFICATION REPORT:
                  precision    recall  f1-score   support

ArmFasterTowards       0.96      0.64      0.76      1142
Ar